In [1]:
import nltk
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
import pprint, time

import sklearn
import sklearn_crfsuite
import scipy.stats
import math, string, re
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from itertools import chain
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn_crfsuite import CRF
from nltk.tokenize import word_tokenize
from nltk.tag.util import untag

import ast
from ast import literal_eval

import jieba 
from hanziconv import HanziConv

from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.ml.feature import Tokenizer
from operator import add

from sklearn.model_selection import train_test_split
from nltk.tokenize import sent_tokenize
import preprocessor as p
import pkuseg

import psycopg2
import pandas as pd
from sqlalchemy import create_engine
from configparser import ConfigParser

In [2]:
f = open("../../Corpus/English/treebank_Mapped.txt", "r")

treebank_Mapped_list = []
for l in f:
    l = literal_eval(l)
    treebank_Mapped_list.append(l)
f.close()

f = open("../../Corpus/English/Brown_Mapped/part-00000", "r")

Brown_Mapped_list = []
for l in f:
    l = literal_eval(l)
    Brown_Mapped_list.append(l)
f.close()

f = open("../../Data/Sentence_Chinese/chinese_postag_sentence.txt", "r")

chinese_postag_sentence = []
for l in f:
    l = literal_eval(l)
    chinese_postag_sentence.append(l)
f.close()

In [3]:
engmal_nltk_list = treebank_Mapped_list + Brown_Mapped_list 
print(len(engmal_nltk_list))

61254


In [4]:
df_Alldata = pd.read_csv('../../Data/Sample_Data/Sample_All.csv')
df_training = pd.read_csv(r'../manuallyTagging.csv')
# df_training = pd.read_csv(r'../testing posTAG.csv')

duplicates = pd.merge(df_Alldata, df_training, how='inner',
                  left_on=['Sentence'], right_on=['f'])

# drop the indices from USERS
df_Alldata = df_Alldata.drop(duplicates.index)

df_Alldata = df_Alldata.drop(['Unnamed: 0', 'Key', 'token_words'], axis = 1)
df_Alldata = df_Alldata.dropna()
print(len(df_Alldata))

178300


In [5]:
df = pd.read_csv(r'../testing posTAG.csv')
df = df.dropna()

In [6]:
df.columns

Index(['f', 'Pos Tag'], dtype='object')

In [7]:
df_withoutchinese = pd.read_csv(r'../testing posTAG.csv')
df_withoutchinese = df_withoutchinese.dropna()

print(len(df_withoutchinese))
counter = 0 
for i in df_withoutchinese['f']:
    if re.findall(r'[\u4e00-\u9fff]', i):
        df_withoutchinese = df_withoutchinese.drop(df_withoutchinese.index[df_withoutchinese['f'] == i])
        # print(i)

print(counter)
print(len(df_withoutchinese))


2404
0
1765


In [8]:
def remove_newline(text):
    return re.sub(r'\n', ' ', str(text))

def find_astrophe(text):
    return bool(re.findall(r'\'', text))

def convert_string2_list(text):

    text = ast.literal_eval(str(text))
    return text

def unicode_problem(text):
    return re.sub(r'[\u0080]','',text).strip()

def unprinable(text):
     return re.sub(r'[\u200B]','',text).strip()
    
def replace_apostrophes(text):
    return re.sub('&#39;|’|´|‘', "'", (str(text)))


# df = pd.read_csv(r'../testing posTAG.csv')
# df = df.dropna()


templist = [] 
templist_withoutchinse = []

for i in df["Pos Tag"]:
    i = remove_newline(i)
    i = unicode_problem(i)
    i = unprinable(i)
    i = replace_apostrophes(i)
    i = i.strip()
    i = convert_string2_list(i)
    
    templist.append(i)

print(len(templist))


for i in df_withoutchinese["Pos Tag"]:
    i = remove_newline(i)
    i = unicode_problem(i)
    i = unprinable(i)
    i = replace_apostrophes(i)
    i = i.strip()
    i = convert_string2_list(i)
    
    templist_withoutchinse.append(i)

print(len(templist_withoutchinse))

2404
1765


In [10]:
# process_data = templist + nltk_list
process_data = templist

process_data_withoutchinese = templist_withoutchinse
print(len(process_data))
print(len(process_data_withoutchinese))

2404
1765


## CRF

In [41]:
def features(sentence, index):
    # """ sentence: [w1, w2, ...], index: the index of the word """
    isChinese = False
    
  
    feature_set = {
        'word': sentence[index],
        'isChinese' : isChinese, 
        'is_first': index == 0,
        'is_last': index == len(sentence) - 1,
        'is_capitalized': sentence[index][0].upper() == sentence[index][0],
        'is_all_caps': sentence[index].upper() == sentence[index],
        'is_all_lower': sentence[index].lower() == sentence[index],
        'prefix-1': sentence[index][0],
        'prefix-2': sentence[index][:2],
        'prefix-3': sentence[index][:3],
        'prefix-4': sentence[index][:4],
        'prefix-5': sentence[index][:5],
        'suffix-1': sentence[index][-1],
        'suffix-2': sentence[index][-2:],
        'suffix-3': sentence[index][-3:],
        'suffix-4': sentence[index][-4:],
        'suffix-5': sentence[index][-5:],
        'prev_word': '' if index == 0 else sentence[index - 1],
        'next_word': '' if index == len(sentence) - 1 else sentence[index + 1],
        'prev2_word': '' if index == 0 else sentence[index - 2],
        'next2_word': '' if index == len(sentence) - 2 or index == len(sentence) - 1  else sentence[index + 2],
        'prev3_word': '' if index == 0 else sentence[index - 3],
        'next3_word': '' if index == len(sentence) - 2 or index == len(sentence) - 1  or index == len(sentence) - 3  else sentence[index + 3],
        'has_hyphen': '-' in sentence[index],
        'is_numeric': sentence[index].isdigit(),
        'capitals_inside': sentence[index][1:].lower() != sentence[index][1:],
        'natural_number': bool(re.findall(r'^[0-9]+', sentence[index])),
        'initcaps' : bool(re.findall(r'^[A-Z]\w+', sentence[index])),
        'initcapsalpha': bool(re.findall(r'^[A-Z][a-z]\w+', sentence[index])),
        'word.stemmed': re.sub(r'(.{2,}?)([aeiougyn]+$)',r'\1', sentence[index].lower()),
        'word.ispunctuation': (sentence[index] in string.punctuation),
        'contain_digit': any(map(str.isdigit, sentence[index])),
        'prev_word_prefix_1': '' if index == 0 else sentence[index - 1][0],
        'prev_word_prefix_2': '' if index == 0 else sentence[index - 1][:2],
        'prev_word_prefix_3': '' if index == 0 else sentence[index - 1][:3],
        'prev_word_prefix_4': '' if index == 0 else sentence[index - 1][:4],
        'next_word_prefix_1': '' if index == len(sentence) - 1 else sentence[index + 1][0] ,
        'next_word_prefix_2': '' if index == len(sentence) - 1 else sentence[index + 1][:2],
        'next_word_prefix_3': '' if index == len(sentence) - 1 else sentence[index + 1][:3],
        'next_word_prefix_4': '' if index == len(sentence) - 1 else sentence[index + 1][:4],
        
         # 'prev_word_suffix_1': '' if index == 0 else sentence[index - 1][-1],
        # 'prev_word_suffix_2': '' if index == 0 else sentence[index - 1][-2],
        # 'prev_word_suffix_3': '' if index == 0 else sentence[index - 1][-3],
        # 'prev_word_suffix_4': '' if index == 0 else sentence[index - 1][-4],
        # 'next_word_suffix_1': '' if index == len(sentence) - 1 else sentence[index + 1][-1],
        # 'next_word_suffix_2': '' if index == len(sentence) - 1 else sentence[index + 1][-2],
        # 'next_word_suffix_3': '' if index == len(sentence) - 1 else sentence[index + 1][-3],
        # 'next_word_suffix_4': '' if index == len(sentence) - 1 else sentence[index + 1][-4]
        
    }               
        # 'formal_word' : 'True' if sentence[index].lower() in lexicon_lower_set else 'False',
           # 'is_shortfrom' : check_shortform_namedentity(sentence[index], 1),
        # 'is_namedentity' : check_shortform_namedentity(sentence[index], 2)


    
    if index <= 0:
        feature_set['BOS'] = True
    
    if index > len(sentence)-1:
        feature_set['EOS'] = True
        
    return feature_set
                      
def transform_to_dataset(tagged_sentences):
    X, y = [], []
 
    for tagged in tagged_sentences:
        X.append([features(untag(tagged), index) for index in range(len(tagged))])
        y.append([tag for _, tag in tagged])
 
    return X, y

def pos_tag(sentence, model, seg):
    sentence = sentence_splitter(sentence, seg)
    sentence_features = [features(sentence, index) for index in range(len(sentence))]
    return list(zip(sentence, model.predict([sentence_features])[0]))

def wordTokenize(sentence):
    
    result = [] 
    nextValue = -1
    punct_start = -1
    doubleTokenize = True
    counter = 0 
    temp_string = ""
    
    combine_punct = bool(re.findall(r'《.*》| “.*”', sentence))
    # print(combine_punct)
    if combine_punct == True: 
        sentence = re.sub(r'(《)','  《' ,sentence)
        sentence = re.sub(r'(》)','》 ' ,sentence)
        sentence = re.sub(r'(“)','  “' ,sentence)
        sentence = re.sub(r'(”)','” ' ,sentence)
    
    text = sentence.split()
    # print(text)
    for index, t in enumerate(text):
        # print(t)
        string = t
        t = t.lower()
 
        if nextValue != -1:
            if nextValue == index:
                continue
            
        if index != 0 and index != (len(text)-1): 
            
            if t == '/':
                
                prevWord = text[index - 1]
                nextWord = text[index + 1]
                
                if prevWord.isdigit() and nextWord.isdigit():
                    string = text[index - 1] + t + text[index + 1]
                    resultlen = len(result)
                    if resultlen != 0:
                        result.pop(resultlen - 1)
            
                    nextValue = index + 1 
                    doubleTokenize = False
                    
        if re.findall(r'^rm\d{1,}', t):
    
            rm = t[:2]
            content = t[2:]
            if re.findall(r'[a-zA-Z]+', content):
                number = re.findall(r'[0-9]+', content)[0]
                temp = re.split(r'[0-9]+', content)
              
                for t in temp:
                    temp_string += t 
                    temp_string += " "
                content = temp_string
                
                string = rm + ' ' + number + ' ' + content
            else: 
                string = rm + ' ' + content
            doubleTokenize = False
        
        if re.findall(r'^[~]+', t):
            if re.findall(r'[a-zA-Z]+', t):
                punctuation = re.findall(r'^[~]+', t)[0]
                word = re.findall(r'[a-zA-Z]+', t)[0]
                
                string = punctuation + ' ' + word
                # print(string)
            doubleTokenize = False
            
    
        if doubleTokenize == True:
        
            if (string.startswith("$") and string.endswith("$")):
                result.append(string)
                
            else:
                nltk_tokenize = word_tokenize(string)
                if len(nltk_tokenize) > 1:
                    for i in nltk_tokenize:
                        result.append(i)
                else:
                    result.append(string)
        else:
            string = string.split()
            for s in string:
                result.append(s)
    
            
    return result

def sentence_splitter(sentence, seg):
    result = []
    sents = wordTokenize(sentence)
    # print(sents)
    for s in sents:
        if re.findall(r'[\u4e00-\u9fff]+', s):
            s = HanziConv.toSimplified(s)
            if (s.startswith(("《")) and s.endswith(("》"))):
                result.append(s)
            elif (s.startswith(("“")) and s.endswith(("”"))):
                result.append(s)
            else:
                result = result + list(seg.cut(s))
        else:
            result.append(s)
    return result


In [40]:
process_data_set = {tuple(sent_list) for sent_list in process_data}

nltk_list_set = {tuple(sent_list) for sent_list in engmal_nltk_list}

cutoff = int(.80 * len(process_data))
training_sentences = random.sample(process_data, k = cutoff)
training_sentence_set = {tuple(sent_list) for sent_list in training_sentences}

test_sentences_set = {}
test_sentences_set = set()

for x in process_data_set:
    if x not in training_sentence_set:
        test_sentences_set.add(x)

# combine_training_set = training_sentence_set.union(nltk_list_set)

X_train, y_train = transform_to_dataset(training_sentence_set)
X_test, y_test = transform_to_dataset(test_sentences_set)

print(cutoff)
print(len(X_train))     
print(len(X_test))    

1923
1847
451


In [14]:
CRF_model_lbfgs = sklearn_crfsuite.CRF(
    algorithm = 'lbfgs',
    max_iterations = 100,
    all_possible_transitions=True,
    c1 = 0.25,
    c2 = 0.35
)
CRF_model_lbfgs.fit(X_train, y_train) 

IOStream.flush timed out
IOStream.flush timed out
/home/pc8/anaconda3/envs/MELex/lib/python3.9/site-packages/sklearn/base.py:209: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '


CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.25, c2=0.35,
    keep_tempfiles=None, max_iterations=100)

In [42]:
CRF_model_lbfgs.fit(X_train, y_train) 

CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.25, c2=0.35,
    keep_tempfiles=None, max_iterations=100)

In [43]:
## with nltk data 
from sklearn_crfsuite import metrics

y_pred_lbfgs = CRF_model_lbfgs.predict(X_test)
print("CRF_model_lbfgs (word level) : ", round(metrics.flat_accuracy_score(y_test, y_pred_lbfgs)*100, 4))
print("CRF_model_lbfgs (sentence level) : ", round(metrics.sequence_accuracy_score(y_test, y_pred_lbfgs)*100, 4))
print("CRF_model_lbfgs    : \n", metrics.flat_classification_report(y_test, y_pred_lbfgs), "\n")

CRF_model_lbfgs (word level) :  81.1594
CRF_model_lbfgs (sentence level) :  37.0288
CRF_model_lbfgs    : 
               precision    recall  f1-score   support

         ADJ       0.87      0.82      0.84       679
         ADP       0.91      0.87      0.89       181
         ADV       0.82      0.75      0.79       353
         AUX       0.45      0.28      0.35        46
       CCONJ       0.95      0.85      0.89        84
         DET       0.87      0.93      0.90       119
        INTJ       0.70      0.41      0.52        39
        NOUN       0.76      0.84      0.80       885
         NUM       0.76      0.82      0.79        67
        PART       0.77      0.80      0.78       111
        PRON       0.98      0.87      0.92       183
       PROPN       0.36      0.55      0.43        82
       PUNCT       0.96      0.98      0.97       456
       SCONJ       0.67      0.62      0.64        39
         SYM       0.20      0.03      0.05        37
        VERB       0.79     

In [45]:
## without  nltk data 

process_data_set = {tuple(sent_list) for sent_list in process_data_withoutchinese}

cutoff = int(.80 * len(process_data))

# training_sentences = process_data[:cutoff]
# test_sentences = process_data[cutoff:]

training_sentences = random.sample(process_data, k = cutoff)
training_sentence_set = {tuple(sent_list) for sent_list in training_sentences}

test_sentences_set = {}
test_sentences_set = set()

for x in process_data_set:
    if x not in training_sentence_set:
        test_sentences_set.add(x)

X_train, y_train = transform_to_dataset(training_sentence_set)
X_test, y_test = transform_to_dataset(test_sentences_set)


CRF_model_lbfgs.fit(X_train, y_train)  

from sklearn_crfsuite import metrics

y_pred_lbfgs_withoutnltk = CRF_model_lbfgs.predict(X_test)
print("CRF_model_lbfgs (word level) : ", round(metrics.flat_accuracy_score(y_test, y_pred_lbfgs_withoutnltk)*100, 4))
print("CRF_model_lbfgs (sentence level) : ", round(metrics.sequence_accuracy_score(y_test, y_pred_lbfgs_withoutnltk)*100, 4))
print("CRF_model_lbfgs    : \n", metrics.flat_classification_report(y_test, y_pred_lbfgs_withoutnltk), "\n")

CRF_model_lbfgs (word level) :  84.7656
CRF_model_lbfgs (sentence level) :  49.1071
CRF_model_lbfgs    : 
               precision    recall  f1-score   support

         ADJ       0.85      0.84      0.84       339
         ADP       0.93      0.90      0.91       163
         ADV       0.86      0.80      0.83       163
         AUX       0.60      0.48      0.54        31
       CCONJ       0.98      0.97      0.98        65
         DET       0.94      0.92      0.93       129
        INTJ       0.50      0.30      0.38        23
        NOUN       0.81      0.88      0.85       536
         NUM       0.83      0.81      0.82        37
        PART       0.39      0.50      0.44        14
        PRON       0.99      0.93      0.96       147
       PROPN       0.44      0.58      0.50        48
       PUNCT       0.98      1.00      0.99       255
       SCONJ       0.00      0.00      0.00         9
         SYM       0.00      0.00      0.00        12
        VERB       0.84     

In [46]:
# Random Search model 

labels = list(CRF_model_lbfgs.classes_)
# labels.remove('O')
print(labels)
params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

# use the same metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score,
                        average='weighted', labels=labels)

# search
rs_crf_lbfgs = RandomizedSearchCV(CRF_model_lbfgs, params_space,
                        cv=3,
                        verbose=1,
                        n_jobs=-1,
                        n_iter=50,
                        scoring=f1_scorer)
rs_crf_lbfgs.fit(X_train, y_train)

['PRON', 'ADV', 'VERB', 'PUNCT', 'AUX', 'NOUN', 'ADJ', 'ADP', 'CCONJ', 'X', 'PROPN', 'DET', 'PART', 'INTJ', 'NUM', 'SCONJ', 'SYM']
Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
/home/pc8/anaconda3/envs/MELex/lib/python3.9/site-packages/sklearn/base.py:209: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '
/home/pc8/anaconda3/envs/MELex/lib/python3.9/site-packages/sklearn/base.py:209: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '
/home/pc8/anaconda3/envs/MELex/lib/python3.9/site-packages/sklearn/base.py:209: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will r

RandomizedSearchCV(cv=3,
                   estimator=CRF(algorithm='lbfgs',
                                 all_possible_transitions=True, c1=0.25,
                                 c2=0.35, keep_tempfiles=None,
                                 max_iterations=100),
                   n_iter=50, n_jobs=-1,
                   param_distributions={'c1': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fa5823e7520>,
                                        'c2': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fa5874a99d0>},
                   scoring=make_scorer(flat_f1_score, average=weighted, labels=['PRON', 'ADV', 'VERB', 'PUNCT', 'AUX', 'NOUN', 'ADJ', 'ADP', 'CCONJ', 'X', 'PROPN', 'DET', 'PART', 'INTJ', 'NUM', 'SCONJ', 'SYM']),
                   verbose=1)

In [ ]:
print('random search best params:', rs.best_params_)
print('random search best CV score:', rs.best_score_)
print('random search model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))


y_pred = rs.predict(X_test)

print("random search CRF_model_lbfgs : ", round(metrics.flat_accuracy_score(y_test, y_pred)*100, 4))
print(" random searchCRF_model_lbfgs : ", round(metrics.sentence_accuracy_score(y_test, y_pred)*100, 4))
print("random search CRF_model_lbfgs    : \n", )
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

